Event based tester - SentDex style 2013 (in SVN Repo)

In [7]:
import pandas as pd
import pandas_datareader as pdr
import seaborn as sns
from matplotlib import pyplot as plt
# not needed, only to prettify the plots.
import matplotlib
import time

# from pandas_datareader import data as pdr
# import pymysql.cursors
import yfinance as yf
# import fix_yahoo_finance as yf     # depredicated
from datetime import datetime
import logging
# import get_id_password_function as auth
import os as os


from IPython.display import set_matplotlib_formats
%matplotlib inline

In [4]:
# http://stackoverflow.com/questions/1557571/how-to-get-time-of-a-python-program-execution
# time execution


# -*- coding: utf-8 -*-
"""
Created on Sun Sep 29 18:10:50 2013
Updated on Sun March 2020

@author: chingl
"""

import time
from datetime import datetime
from time import mktime

start_time = time.time()

In [2]:
#Trade Rules:
#Buy every time you are not invested and stock price drops.
#sell every time the stock price is more than .2% higher than when you bought. (.002)

    investmentAmount = 50000
    tradeComm = 7

    stance = 'none'
    buyPrice = 0
    sellPrice = 0
    previousPrice = 0

    totalProfit = 0

    tradeCount = 0
    startingPrice = 0

    startingTime = 0
    endingTime = 0

    totalInvestedTime = 0

    overallStartingTime = 0
    overallEndingTime = 0

In [5]:
# this worked
tickerList = ['VAS.AX','VHY.AX','VGB.AX']
for ticker in tickerList:
    df = pdr.get_data_yahoo(ticker, start=datetime(2001,1,1), end=datetime(2020,3,19))
    file_path = '/home/tester/GitHub/AlgoTrading/notebook/'
    df.to_csv( file_path + ticker + '.csv')

In [18]:
# read the file
ticker = 'VHY.AX'
file_path = '/home/tester/GitHub/AlgoTrading/notebook/'
# for ticker in tickerList:
df = pd.read_csv( file_path + ticker + '.csv', index_col=None, encoding = "ISO-8859-1", engine='python')  # create a pandas df

In [29]:
df.tail()

,Date,High,Low,Open,Close,Volume,Adj Close
2190,2020-03-12,47.750000,42.320000,45.000000,47.180000,224471.0,47.180000
2191,2020-03-15,47.330002,44.590000,47.330002,45.070000,130056.0,45.070000
2192,2020-03-16,47.000000,43.500000,45.000000,46.959999,263387.0,46.959999
2193,2020-03-17,47.000000,43.349998,47.000000,44.450001,143623.0,44.450001
2194,2020-03-18,45.919998,42.500000,44.630001,42.619999,93423.0,42.619999


In [44]:
# The tester logic: 

#Trade Rules:
#Buy every time you are not invested and stock price drops.
#sell every time the stock price is more than .2% higher than when you bought. (.002)
# this is use per unit price for testing

investmentAmount = 5000
tradeComm = 7

stance = 'none'
buyPrice = 0
sellPrice = 0
previousPrice = 0

totalProfit = 0

tradeCount = 0
startingPrice = 0

startingTime = 0
endingTime = 0

totalInvestedTime = 0

overallStartingTime = 0
overallEndingTime = 0
    
    
for index, row in df.iterrows(): 
    # print(' index is:', index)
    stockPrice = row['Close']
    # Debug to verify that prices are processed
    # print(stockPrice, '----', previousPrice)
    if stance == 'none':
        #print stockPrice, previousPrice
        # if stockPrice < (previousPrice-previousPrice*0.1):  # gross and net profits divided by zero
        if stockPrice < (previousPrice):  # gross and net profits divided by zero
            print('buy triggered!')
            buyPrice = stockPrice
            print('bought stock for',buyPrice)
            stance = 'holding'
            print('The buy date is: ', row['Date'])
        
            #### startingTime = unixStamp
        
            if tradeCount == 0:
                startingPrice = buyPrice
                #### overallStartingTime = unixStamp
                tradeCount += 1
    
    elif stance == 'holding':
        if stockPrice > (buyPrice * .01 + buyPrice):
            print('sell triggered!')
            sellPrice = stockPrice
            print('finished trade, sold for:',sellPrice)
            stance = 'none'
            print('The sell date is: ', row['Date'])  
            tradeProfit = sellPrice - buyPrice
            totalProfit += tradeProfit
            print('The profile of the trade is: ', totalProfit, '\n') # line to mark the end of a round trip
            #### endingTime = unixStamp
            #### timeInvested = endingTime - startingTime
            #### totalInvestedTime += timeInvested
        
            #### overallEndingTime = endingTime
            tradeCount += 1
            
    # if reached the end of data frame, then close the trade
        if index == 2194:
            print('sell triggered at end of data frame!')
            sellPrice = stockPrice
            print('finished trade, sold for:',sellPrice)
            stance = 'none'
            print('The sell date is: ', row['Date'])  
            tradeProfit = sellPrice - buyPrice
            totalProfit += tradeProfit
            print('The profit of the trade is: ', totalProfit, '\n') # line to mark the end of a round trip
            #### endingTime = unixStamp
            #### timeInvested = endingTime - startingTime
            #### totalInvestedTime += timeInvested
        
            #### overallEndingTime = endingTime
            tradeCount += 1
           
    previousPrice = stockPrice   # this is a tricky mistake, it should be indented under the main loop

print('\n','Trading Complete.')
print('Profit Per Stock:', totalProfit)
print('Trade Count:', tradeCount)
#### grossPercProf = totalProfit/startingPrice * 100     # divided by zero if buy condition *0.02
#### print('Gross % Profit:', grossPercProf)
    
print('How much gross profit did we make off of our',investmentAmount,'investment:')
grossProf = investmentAmount/startingPrice*totalProfit
print('we made:', grossProf)
tradeCommsTotal = tradeComm * tradeCount
print('amount paid in commissions:',tradeCommsTotal)
netProf = grossProf-tradeCommsTotal
print('profit after commissions:',netProf)
netPercentProf = netProf/investmentAmount * 100
print('net percent profit after comms:',netPercentProf)

'''
averageInvestedTimeHours = totalInvestedTime/tradeCount/3600
print('average holding length in hours:', averageInvestedTimeHours)
    
overallTime = overallEndingTime - overallStartingTime
percentageInvested = totalInvestedTime/overallTime * 100
    
print('cumulative amount of time holding in days: ',overallTime/86400)
print('Percentage of the time invested:',percentageInvested)  
'''  
        
        

buy triggered!
bought stock for 50.47999954223633
The buy date is:  2011-06-01
sell triggered!
finished trade, sold for: 51.2400016784668
The sell date is:  2012-08-17
The profile of the trade is:  0.7600021362304688 

buy triggered!
bought stock for 51.9000015258789
The buy date is:  2012-08-21
sell triggered!
finished trade, sold for: 52.66999816894531
The sell date is:  2012-10-16
The profile of the trade is:  1.529998779296882 

buy triggered!
bought stock for 52.790000915527344
The buy date is:  2012-10-18
sell triggered!
finished trade, sold for: 53.4900016784668
The sell date is:  2012-12-02
The profile of the trade is:  2.2299995422363352 

buy triggered!
bought stock for 53.34000015258789
The buy date is:  2012-12-03
sell triggered!
finished trade, sold for: 53.97999954223633
The sell date is:  2012-12-10
The profile of the trade is:  2.8699989318847727 

buy triggered!
bought stock for 53.9000015258789
The buy date is:  2012-12-12
sell triggered!
finished trade, sold for: 54.

"\naverageInvestedTimeHours = totalInvestedTime/tradeCount/3600\nprint('average holding length in hours:', averageInvestedTimeHours)\n    \noverallTime = overallEndingTime - overallStartingTime\npercentageInvested = totalInvestedTime/overallTime * 100\n    \nprint('cumulative amount of time holding in days: ',overallTime/86400)\nprint('Percentage of the time invested:',percentageInvested)  \n"

In [49]:
# The tester logic: 

#Trade Rules:
#Buy every time you are not invested and stock price drops.
#sell every time the stock price is more than .2% higher than when you bought. (.002)
# this is use per WHOLE amount for testing

investmentAmount = 5000
balance = 0
tradeComm = 7

stance = 'none'
buyPrice = 0
sellPrice = 0
previousPrice = 0

totalProfit = 0

tradeCount = 0
startingPrice = 0

startingTime = 0
endingTime = 0

totalInvestedTime = 0

overallStartingTime = 0
overallEndingTime = 0

capital_gain = 0

    
    
for index, row in df.iterrows():
    if balance == 0:
        capital = investmentAmount
    elif balance > 0:
        capital = balance
        
    # print(' index is:', index)
    stockPrice = row['Close']
    # Debug to verify that prices are processed
    # print(stockPrice, '----', previousPrice)
    if stance == 'none':
        #print stockPrice, previousPrice
        # if stockPrice < (previousPrice-previousPrice*0.1):  # gross and net profits divided by zero
        if stockPrice < (previousPrice):  # gross and net profits divided by zero
            print('buy triggered!')
            buyPrice = stockPrice
            unit = capital / float(buyPrice)
            print('bought stock for',buyPrice)
            stance = 'holding'
            print('The buy date is: ', row['Date'])
        
            #### startingTime = unixStamp
        
            if tradeCount == 0:
                startingPrice = buyPrice
                #### overallStartingTime = unixStamp
                tradeCount += 1
    
    elif stance == 'holding':
        if stockPrice > (buyPrice * .03 + buyPrice) or stockPrice < (-buyPrice * .01 + buyPrice):
            print('sell triggered!')
            sellPrice = stockPrice
            print('finished trade, sold for:',sellPrice)
            stance = 'none'
            print('The sell date is: ', row['Date'])  
            tradeProfit = sellPrice - buyPrice
            totalProfit += tradeProfit
            print('The profile of the trade is: ', totalProfit, '\n') # line to mark the end of a round trip
            capital = unit * float(sellPrice)
            capital_gain = unit*(float(sellPrice) - float(buyPrice))
            balance = unit*(float(sellPrice))
            #### endingTime = unixStamp
            #### timeInvested = endingTime - startingTime
            #### totalInvestedTime += timeInvested
        
            #### overallEndingTime = endingTime
            tradeCount += 1
        print("capital gain is: ", capital_gain)
        print("balance is: ", balance)
            
    # if reached the end of data frame, then close the trade
        if index == 2194:
            print('sell triggered at end of data frame!')
            sellPrice = stockPrice
            print('finished trade, sold for:',sellPrice)
            stance = 'none'
            print('The sell date is: ', row['Date'])  
            tradeProfit = sellPrice - buyPrice
            totalProfit += tradeProfit
            print('The profit of the trade is: ', totalProfit, '\n') # line to mark the end of a round trip
            #### endingTime = unixStamp
            #### timeInvested = endingTime - startingTime
            #### totalInvestedTime += timeInvested
        
            #### overallEndingTime = endingTime
            tradeCount += 1
           
    previousPrice = stockPrice   # this is a tricky mistake, it should be indented under the main loop

print('\n','Trading Complete.')
print('Profit Per Stock:', totalProfit)
print('Trade Count:', tradeCount)
#### grossPercProf = totalProfit/startingPrice * 100     # divided by zero if buy condition *0.02
#### print('Gross % Profit:', grossPercProf)
    
print('How much gross profit did we make off of our',investmentAmount,'investment:')
grossProf = investmentAmount/startingPrice*totalProfit
print('we made:', grossProf)
tradeCommsTotal = tradeComm * tradeCount
print('amount paid in commissions:',tradeCommsTotal)
netProf = grossProf-tradeCommsTotal
print('profit after commissions:',netProf)
netPercentProf = netProf/investmentAmount * 100
print('net percent profit after comms:',netPercentProf)

'''
averageInvestedTimeHours = totalInvestedTime/tradeCount/3600
print('average holding length in hours:', averageInvestedTimeHours)
    
overallTime = overallEndingTime - overallStartingTime
percentageInvested = totalInvestedTime/overallTime * 100
    
print('cumulative amount of time holding in days: ',overallTime/86400)
print('Percentage of the time invested:',percentageInvested)  
'''  
        

buy triggered!
bought stock for 50.47999954223633
The buy date is:  2011-06-01
sell triggered!
finished trade, sold for: 49.22000122070313
The sell date is:  2011-06-02
The profile of the trade is:  -1.259998321533196 

capital gain is:  -124.80173662432014
balance is:  4875.19826337568
buy triggered!
bought stock for 49.04999923706055
The buy date is:  2011-06-06
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -124.80173662432014
balance is:  4875.19826337568
capital gain is:  -12

The buy date is:  2012-05-02
capital gain is:  155.8138096319276
balance is:  4800.777158939453
capital gain is:  155.8138096319276
balance is:  4800.777158939453
sell triggered!
finished trade, sold for: 49.41999816894531
The sell date is:  2012-05-07
The profile of the trade is:  -2.8100013732910156 

capital gain is:  -93.34876169071121
balance is:  4707.428397248742
buy triggered!
bought stock for 49.22999954223633
The buy date is:  2012-05-09
capital gain is:  -93.34876169071121
balance is:  4707.428397248742
capital gain is:  -93.34876169071121
balance is:  4707.428397248742
capital gain is:  -93.34876169071121
balance is:  4707.428397248742
capital gain is:  -93.34876169071121
balance is:  4707.428397248742
capital gain is:  -93.34876169071121
balance is:  4707.428397248742
sell triggered!
finished trade, sold for: 48.31999969482422
The sell date is:  2012-05-17
The profile of the trade is:  -3.720001220703125 

capital gain is:  -87.01521761186657
balance is:  4620.413179636876

The buy date is:  2013-04-26
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  243.18277712782321
balance is:  5985.158921836017
capital gain is:  2

capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital gain is:  196.45109314667712
balance is:  6222.1910507019
capital ga

capital gain is:  -63.836349964483595
balance is:  5894.527705446885
sell triggered!
finished trade, sold for: 66.95999908447266
The sell date is:  2015-05-06
The profile of the trade is:  7.799999237060661 

capital gain is:  -103.7772865243403
balance is:  5790.750418922545
buy triggered!
bought stock for 66.5
The buy date is:  2015-05-07
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance is:  5790.750418922545
capital gain is:  -103.7772865243403
balance 

...

balance is:  5013.460481875357
capital gain is:  -116.39476064081373
balance is:  5013.460481875357
capital gain is:  -116.39476064081373
balance is:  5013.460481875357
capital gain is:  -116.39476064081373
balance is:  5013.460481875357
sell triggered!
finished trade, sold for: 60.47000122070313
The sell date is:  2018-02-25
The profile of the trade is:  1.1500053405762714 

capital gain is:  161.7520818469463
balance is:  5175.2125637223035
buy triggered!
bought stock for 60.15999984741211
The buy date is:  2018-02-27
capital gain is:  161.7520818469463
balance is:  5175.2125637223035
sell triggered!
finished trade, sold for: 59.52000045776367
The sell date is:  2018-03-01
The profile of the trade is:  0.5100059509278339 

capital gain is:  -55.05540044022585
balance is:  5120.157163282078
buy triggered!
bought stock for 59.04999923706055
The buy date is:  2018-03-04
capital gain is:  -55.05540044022585
balance is:  5120.157163282078
capital gain is:  -55.05540044022585
balance is:  

capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:  5085.307300149038
capital gain is:  -57.57260158929238
balance is:

capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:  5549.830328590506
capital gain is:  190.56778781688348
balance is:

buy triggered!
bought stock for 45.06999969482422
The buy date is:  2020-03-15
sell triggered!
finished trade, sold for: 46.959999084472656
The sell date is:  2020-03-16
The profile of the trade is:  -5.6199874877929545 

capital gain is:  184.3832564144061
balance is:  4581.291189741181
buy triggered!
bought stock for 44.45000076293945
The buy date is:  2020-03-17
sell triggered!
finished trade, sold for: 42.61999893188477
The sell date is:  2020-03-18
The profile of the trade is:  -7.449989318847635 

capital gain is:  -188.611273834017
balance is:  4392.679915907164
sell triggered at end of data frame!
finished trade, sold for: 42.61999893188477
The sell date is:  2020-03-18
The profit of the trade is:  -9.279991149902315 


 Trading Complete.
Profit Per Stock: -9.279991149902315
Trade Count: 214
How much gross profit did we make off of our 5000 investment:
we made: -919.1750429927995
amount paid in commissions: 1498
profit after commissions: -2417.1750429927997
net percent profit a

"\naverageInvestedTimeHours = totalInvestedTime/tradeCount/3600\nprint('average holding length in hours:', averageInvestedTimeHours)\n    \noverallTime = overallEndingTime - overallStartingTime\npercentageInvested = totalInvestedTime/overallTime * 100\n    \nprint('cumulative amount of time holding in days: ',overallTime/86400)\nprint('Percentage of the time invested:',percentageInvested)  \n"

In [3]:
# http://stackoverflow.com/questions/1557571/how-to-get-time-of-a-python-program-execution
# time execution


# -*- coding: utf-8 -*-
"""
Created on Sun Sep 29 18:10:50 2013
Updated on Sun March 2020

@author: chingl
"""

import time
from datetime import datetime
from time import mktime

start_time = time.time()

def backTest():


#Trade Rules:
#Buy every time you are not invested and stock price drops.
#sell every time the stock price is more than .2% higher than when you bought. (.002)

    investmentAmount = 50000
    tradeComm = 7

    stance = 'none'
    buyPrice = 0
    sellPrice = 0
    previousPrice = 0

    totalProfit = 0

    tradeCount = 0
    startingPrice = 0

    startingTime = 0
    endingTime = 0

    totalInvestedTime = 0

    overallStartingTime = 0
    overallEndingTime = 0

    bigDataFile = open('AMAT.txt','r')
    readFile = bigDataFile.read()
    lineSplit = readFile.split('\n')

    for everyLine in lineSplit:
    
        dividedLine = everyLine.split(',')
        stockName = dividedLine[0].split('.')[1]
        initialDate = dividedLine[2]+dividedLine[3]
        # print initialDate 
        datetime.strptime(initialDate, '%Y%m%d%H%M%S').timetuple() 
        unixStamp = mktime(datetime.strptime(initialDate, '%Y%m%d%H%M%S').timetuple())
        # print unixStamp
        dateStamp = time.strftime('%m/%d/%Y %H:%M:%S',time.localtime(unixStamp))
        
        #print dateStamp
        stockPrice = float(dividedLine[4])
        reformatted = unixStamp,dateStamp,stockName,stockPrice
        # print reformatted
    
        if stance == 'none':
            #print stockPrice, previousPrice
            if stockPrice < previousPrice:
    
                print('buy triggered!')
                buyPrice = stockPrice
                print('bought stock for',buyPrice)
                stance = 'holding'
        
                startingTime = unixStamp
        
                if tradeCount == 0:
    
                    startingPrice = buyPrice
                    overallStartingTime = unixStamp
    
                tradeCount += 1
    
        elif stance == 'holding':
            if stockPrice > buyPrice * .002 + buyPrice:
                print('sell triggered!')
                sellPrice = stockPrice
                print('finished trade, sold for:',sellPrice)
                stance = 'none'
                tradeProfit = sellPrice - buyPrice
                totalProfit += tradeProfit
                print(totalProfit)
                endingTime = unixStamp
                timeInvested = endingTime - startingTime
                totalInvestedTime += timeInvested
        
                overallEndingTime = endingTime
        
                tradeCount += 1
    
        previousPrice = stockPrice   # this is a tricky mistake, it should be indented under the main loop
 
      
    print('Trading Complete.')
    print('Profit Per Stock:', totalProfit)
    print('Trade Count:', tradeCount)
    grossPercProf = totalProfit/startingPrice * 100
    print('Gross % Profit:', grossPercProf)
    
    print('How much gross profit did we make off of our',investmentAmount,'investment:')
    grossProf = investmentAmount/startingPrice*totalProfit
    print('we made:', grossProf)
    tradeCommsTotal = tradeComm * tradeCount
    print('amount paid in commissions:',tradeCommsTotal)
    netProf = grossProf-tradeCommsTotal
    print('profit after commissions:',netProf)
    netPercentProf = netProf/investmentAmount * 100
    print('net percent profit after comms:',netPercentProf)
    
    averageInvestedTimeHours = totalInvestedTime/tradeCount/3600
    
    print('average holding length in hours:', averageInvestedTimeHours)
    
    overallTime = overallEndingTime - overallStartingTime
    percentageInvested = totalInvestedTime/overallTime * 100
    
    print('cumulative amount of time holding in days: ',overallTime/86400)
    print('Percentage of the time invested:',percentageInvested)
 
backTest()

print('execution time is: ', time.time() - start_time, " seconds")
print('execution time is %s seconds ' %(time.time() - start_time)) 

buy triggered!
bought stock for 11.67
sell triggered!
finished trade, sold for: 11.71
0.040000000000000924
buy triggered!
bought stock for 11.7
sell triggered!
finished trade, sold for: 11.725
0.06500000000000128
buy triggered!
bought stock for 11.72
sell triggered!
finished trade, sold for: 11.75
0.09500000000000064
buy triggered!
bought stock for 11.74
sell triggered!
finished trade, sold for: 11.77
0.125
buy triggered!
bought stock for 11.76
sell triggered!
finished trade, sold for: 11.79
0.15499999999999936
buy triggered!
bought stock for 11.785
sell triggered!
finished trade, sold for: 11.81
0.17999999999999972
buy triggered!
bought stock for 11.805
sell triggered!
finished trade, sold for: 11.83
0.20500000000000007
buy triggered!
bought stock for 11.825
sell triggered!
finished trade, sold for: 11.85
0.23000000000000043
buy triggered!
bought stock for 11.845
sell triggered!
finished trade, sold for: 11.87
0.254999999999999
buy triggered!
bought stock for 11.875
sell triggered!
fi

sell triggered!
finished trade, sold for: 14.15
2.4999999999999964
buy triggered!
bought stock for 14.155
sell triggered!
finished trade, sold for: 14.185
2.5299999999999976
buy triggered!
bought stock for 14.18
sell triggered!
finished trade, sold for: 14.21
2.5599999999999987
buy triggered!
bought stock for 14.2
sell triggered!
finished trade, sold for: 14.23
2.59
buy triggered!
bought stock for 14.22
sell triggered!
finished trade, sold for: 14.25
2.619999999999999
buy triggered!
bought stock for 14.245
sell triggered!
finished trade, sold for: 14.275
2.6500000000000004
buy triggered!
bought stock for 14.27
sell triggered!
finished trade, sold for: 14.3
2.6800000000000015
buy triggered!
bought stock for 14.3
sell triggered!
finished trade, sold for: 14.33
2.710000000000001
buy triggered!
bought stock for 14.335
sell triggered!
finished trade, sold for: 14.365
2.74
buy triggered!
bought stock for 14.36
sell triggered!
finished trade, sold for: 14.39
2.7700000000000014
buy triggered!
